In [1]:
import os
import numpy as np
import keras
from keras.models import model_from_json
from skimage.transform import resize
from skimage.io import imread
from keras.utils.np_utils import to_categorical
from random import seed, sample
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
json_file = open('model_5_5.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_5_5.h5")
print("Loaded model from disk")

FileNotFoundError: ignored

In [0]:
!tar -xvf journal.tar.gz

In [0]:
rootDir = '.'
n_images = 10000
images = np.zeros((n_images,) + (20, 20))

labels = []
lab = {
    '0': 1, '1': 2, '2': 3, '3':4, '4':5, '5':6, '6':7, '7':8, '8':9, '9':10,
    'A': 11, 'B': 12, 'C': 13, 'D': 14, 'E': 15, 'F': 16, 'G': 17, 'H': 18, 'I': 19, 'J': 20, 'K': 21, 'L': 22, 'M': 23, 
    'N': 24, 'O': 25, 'P': 26, 'Q': 27, 'R': 28, 'S': 29, 'T': 30, 'U': 31, 'V': 32, 'W': 33, 'X': 34, 'Y': 35, 'Z': 36,
    'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 
    'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'x': 34, 'y': 35, 'z': 36
}
j = 0
for dirName, subdirList, fileList in os.walk(rootDir):
    # print(dirName, subdirList, fileList)
    i = 0
    for fname in fileList:
        if 'png' not in fname:
            continue
        full_path = os.path.join(dirName, fname)
        
        if dirName[-1] in lab and dirName[-2] == '/':
            print(full_path, lab[dirName[-1]])
        else:
            continue
        
        images[j] = resize(imread(full_path, as_grey=True), (20, 20)) 
        labels.append(lab[dirName[-1]])
        j += 1
        i += 1
        # if i == 5:
        #     break
images = images[:len(labels)]

In [0]:
x = images.reshape(images.shape[0], 20, 20 , 1).astype('float32')
y = to_categorical(labels, num_classes=37)

In [0]:
len(labels), images[0], labels[0]

In [37]:
labels[0], y[0][28]

(28, 1.0)

In [8]:
loaded_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
score = loaded_model.evaluate(x, y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

acc: 8.47%


In [30]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        51264     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               262272    
__________

In [31]:
print(len(loaded_model.layers))
for layer in loaded_model.layers[:-1]:
    layer.trainable = False

9


In [32]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        51264     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               262272    
__________

In [41]:
loaded_model.layers.pop()

In [42]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        51264     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
Total params: 257,024
Trainable params: 0
Non-trainable params: 257,024
_________________________________________________________________


In [0]:
num_classes = 36
loaded_model.add(Dense(num_classes, activation='softmax'))
loaded_model.layers[-1].name += '_2'


In [44]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        51264     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_5_2 (Dense)            (None, 36)                1332      
Total para

In [48]:
k = sample(range(len(images)), len(images))
im_shuf = images[k]
labels_shuf = np.array(labels)[k]
    
ocr = {
    'images': im_shuf,
    'data': im_shuf.reshape((im_shuf.shape[0], -1)), # / 255.0
    'target': labels_shuf
}
x_train, x_test, y_train, y_test = train_test_split(im_shuf, labels_shuf, random_state=2, train_size=0.8)
X_train = x_train.reshape(x_train.shape[0], 20, 20 , 1).astype('float32')
X_test = x_test.reshape(x_test.shape[0], 20, 20 , 1).astype('float32')
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [14]:
labels_shuf

array([11, 30, 24, ..., 35, 29, 11])

In [49]:
batch_size = 128
loaded_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
loaded_model.fit(X_train, y_train, batch_size=batch_size,
                        nb_epoch=60,
                        verbose=1)

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.5821 - acc: 0.0430
Epoch 2/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.5576 - acc: 0.0742
Epoch 3/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.5341 - acc: 0.0795
Epoch 4/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.5115 - acc: 0.0742
Epoch 5/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.4897 - acc: 0.0742
Epoch 6/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.4686 - acc: 0.0742
Epoch 7/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.4484 - acc: 0.0742
Epoch 8/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.4290 - acc: 0.0742
Epoch 9/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.4102 - acc: 0.0742
Epoch 10/60
1510/1510 [==============================] - 2s 1ms/step - loss: 3.3921 - acc: 0.0742
Epoch 11/60
1510/1510 [======

In [46]:
num_classes = 36
batch_size = 128

X_train = x_train.reshape(x_train.shape[0], 20, 20 , 1).astype('float32')
X_test = x_test.reshape(x_test.shape[0], 20, 20 , 1).astype('float32')
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(20, 20, 1)))

model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.summary()
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size,
                        nb_epoch=60,
                        verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 18, 18, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
Total params: 92,672
Trainable params: 92,672
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: ignored

In [78]:
scores = model.evaluate(X_test, y_test, verbose = 10 )
print ( scores )

[2.455313619482454, 0.5529100530677371]
